In [21]:
import sys
sys.path.insert(0, '..')
import models_neural.quote_attribution.classification_models as cm 
import json
import attrdict
from models_neural.src.config_helper import TransformersConfig
import pandas as pd 
import zipfile
import models_neural.quote_attribution.utils_dataset as ud

In [22]:
from importlib import reload

In [23]:
reload(ud)

<module 'models_neural.quote_attribution.utils_dataset' from '../models_neural/quote_attribution/utils_dataset.py'>

In [24]:
SourceQA, SourceClassifier = cm.SourceQA, cm.SourceClassifier
SourceQADataModule, SourceClassificationDataModule = ud.SourceQADataModule, ud.SourceClassificationDataModule

In [25]:
remote = False
if remote:
    data_fn = 'our-annotated-data__stage-2.tsv'
    config_out = 'config-%s.json' % "Stage 2: Quote Attribution. Our dataset only."
else:
    data_fn = '../models_neural/quote_attribution/data/our-annotated-data__stage-2.tsv'
    config_out = '../models_neural/quote_attribution/config-sentence-classification.json'
data_df = pd.read_csv(data_fn, sep='\t', header=None)

In [26]:
with open(config_out) as f:
    config_dict = json.load(f)

config_dict = attrdict.AttrDict(config_dict)
config_dict.pretrained_files_s3 = config_dict.pretrained_model_path
config_dict.model_type = 'roberta'
config_dict.shuffle_data = False

In [89]:
config_dict.downsample_negative_data = 1

In [94]:
config_dict.num_documents = None

In [95]:
data_model = SourceClassificationDataModule(
    config=config_dict,
    data_fp=data_fn,
    model_type='roberta',
    max_length_seq=2048,
    pretrained_model_path=config_dict.pretrained_model_path,
    batch_size=1,
    spacy_path='en_core_web_lg'
)

In [96]:
dataset = data_model.get_dataset()

  0%|          | 0/335 [00:00<?, ?it/s]

not found Pacific Tsunami Warning Center, test...
not found The Sydney Morning Herald, test...
not found Río Paya cooperative, test...
not found Peninsula , test...
not found “ Today ” show, test...
not found ICE, test...
not found Canesius Ndayimanisha, test...
not found MoMA PS1, test...


In [97]:
len(dataset)

29906

In [101]:
pd.Series(list(map(lambda x: x['label'], dataset))).value_counts()

True     14953
False    14953
dtype: int64

In [107]:
import unidecode
data_df[0].apply(unidecode.unidecode).pipe(lambda s: s == '"').value_counts()

False    13697
True      2096
Name: 0, dtype: int64

In [113]:
t = pd.DataFrame(dataset.input).loc[lambda df: df['label'] == True]

In [116]:
t['doc_tokens'].drop_duplicates()

0        [0, 5226, 18718, 111, 2236, 163, 9298, 530, 93...
53       [0, 5226, 18718, 111, 2236, 6823, 23791, 565, ...
130      [0, 5226, 18718, 111, 2236, 83, 9823, 14, 5, 1...
173      [0, 5226, 18718, 111, 2236, 1261, 16, 4568, 13...
286      [0, 5226, 18718, 111, 2236, 15123, 1889, 387, ...
                               ...                        
29601    [0, 5226, 18718, 111, 2236, 18683, 5809, 38, 6...
29632    [0, 5226, 18718, 111, 2236, 38, 1301, 448, 521...
29684    [0, 5226, 18718, 111, 2236, 96, 5, 2625, 81, 2...
29736    [0, 5226, 18718, 111, 2236, 20, 2903, 539, 34,...
29872    [0, 5226, 18718, 111, 2236, 7135, 21850, 2156,...
Name: doc_tokens, Length: 322, dtype: object

In [124]:
tokenizer.decode(t['doc_tokens'].drop_duplicates().iloc[2])

'<s> journalist passive - voice A firearm that the authorities said was used on Sunday in a mass shooting at a nightclub in Orlando, Fla., is a descendant of one of the world</s><s> ’s most widely distributed and familiar infantry weapons, and a type of rifle that has been involved in previous mass shootings in the United States.</s><s> Chief John Mina of the Orlando Police Department said the gunman ’s weapons included a handgun and an “ AR-15 - type assault rifle.</s><s> ”</s><s> The first AR-15s were designed in the 1950s by Eugene M. Stoner, a Marine and inventor, who developed the weapon to military standards and for military service.</s><s> It was an atypical rifle for its time, seemingly futuristic, and made partly with lightweight plastics and aluminum that traditionalists scorned.</s><s> It fired a small - caliber, high - velocity bullet — the.223</s><s> — that was also considered revolutionary.</s><s> The rifle was capable, via a selector lever, of semiautomatic or automatic 

In [125]:
t

,source_ind_tokens,sentence_ind_tokens,doc_tokens,label,doc_idx,sent_idx,annotated_source,candidate_source
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...","[0, 5226, 18718, 111, 2236, 163, 9298, 530, 93...",True,/train/902,NaN,None,None
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 5226, 18718, 111, 2236, 163, 9298, 530, 93...",True,/train/902,NaN,None,None
6,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 5226, 18718, 111, 2236, 163, 9298, 530, 93...",True,/train/902,NaN,None,None
7,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 5226, 18718, 111, 2236, 163, 9298, 530, 93...",True,/train/902,NaN,None,None
8,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 5226, 18718, 111, 2236, 163, 9298, 530, 93...",True,/train/902,NaN,None,None
...,...,...,...,...,...,...,...,...
29899,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 5226, 18718, 111, 2236, 7135, 21850, 2156,...",True,/train/716,NaN,Elizabeth Holmes,Elizabeth Holmes
29900,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 5226, 18718, 111, 2236, 7135, 21850, 2156,...",True,/train/716,NaN,Elizabeth Holmes,Elizabeth Holmes
29902,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 5226, 18718, 111, 2236, 7135, 21850, 2156,...",True,/train/716,NaN,company spokesman,company spokesman
29903,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 5226, 18718, 111, 2236, 7135, 21850, 2156,...",True,/train/716,NaN,None,None


In [36]:
true_examples = list(filter(lambda x: x['label'] == True, data_model.test_dataset))

In [40]:
from transformers import AutoTokenizer

In [41]:
tokenizer= AutoTokenizer.from_pretrained('/Users/alex/.cache/torch/transformers/named-models/roberta-base-expanded-embeddings')

In [42]:
tokenizer.decode(data_model.test_dataset[0]['doc_tokens'])

'<s> journalist passive - voice In a rare display of internal strife, the opinion department of The Wall Street Journal openly antagonized the paper ’s news staff Thursday by publishing a tersely worded note to readers just days after it found itself on the receiving end of a sharply worded critique signed by hundreds of newsroom employees.</s><s> Turmoil inside the buttoned - up newsroom had been brewing for more than a month before the clash spilled into the view of the paper ’s readership.</s><s> The latest skirmish started Tuesday, when nearly 300 of The Journal ’s news staff members sent a sharply worded letter to the paper ’s publisher, Almar Latour, condemning the opinion desk</s><s> ’s</s><s> “ lack of fact - checking and transparency.</s><s> ”</s><s> The letter cited several examples of essays published by the opinion section, which is operated separately from the newsroom, that included factual errors, among them a June 16 article by Vice President Mike Pence.</s><s> The essa

In [85]:
pd.DataFrame(true_examples)['doc_idx'].unique()

array(['/test/825', '/test/561', '/test/589', '/test/931', '/test/580',
       '/test/381', '/test/377', '/test/803', '/test/560', '/test/497',
       '/test/460', '/test/946', '/test/929', '/test/27', '/test/597',
       '/test/849', '/test/496', '/test/31', '/test/590', '/test/10',
       '/test/388', '/test/907', '/test/824', '/test/16', '/test/32',
       '/test/903', '/test/498', '/test/581', '/test/393', '/test/454',
       '/test/488'], dtype=object)

In [48]:
true_examples[0]['source_ind_tokens']

6

In [53]:
true_examples_with_zero_source = list(filter(lambda x: sum(x['source_ind_tokens'])== 0, true_examples))

In [57]:
true_examples_with_zero_source[0]['annotated_source']

'None'

In [60]:
true_examples_with_zero_source[0]['candidate_source']

'None'

In [74]:
pd.concat([
    pd.Series(true_examples_with_zero_source[0]['doc_tokens']),
    pd.Series(true_examples_with_zero_source[0]['sentence_ind_tokens']),
], axis=1).loc[lambda df: df[1] == 1][0].apply(tokenizer.decode)

1151               <s>
1152     Revolutionary
1153              acts
1154              come
1155                at
1156                 a
1157              cost
1158                 .
1159              </s>
Name: 0, dtype: object

In [75]:
len(true_examples)

150

In [78]:
data_df.loc[lambda df: df[3].str.contains('/test/')].assign(c=1).groupby([1, 3])['c'].value_counts()

1                                 3          c
\tBambang Soelistyo               /test/776  1    2
\tEnvironmental officials         /test/172  1    1
 Federal Aviation Administration  /test/497  1    1
2010 study                        /test/27   1    2
2011 United Nations report        /test/597  1    1
                                                 ..
white house fact sheet            /test/448  1    1
witnesses                         /test/404  1    1
woman                             /test/10   1    1
Édouard Philippe                  /test/789  1    1
“ Today ” show                    /test/549  1    8
Name: c, Length: 783, dtype: int64

In [29]:
data_df.loc[lambda df: df[3].str.contains('/test/')]

,0,1,2,3
162,China is canceling plans to build more than 10...,China,0,/test/903
163,"The announcement , made by China ’s National E...",China ’s National Energy Administration,1,/test/903
164,That includes dozens of projects in 13 provinc...,China ’s National Energy Administration,2,/test/903
165,Those projects alone would have had a combined...,Greenpeace,3,/test/903
166,The cancellations make it likelier that China ...,None,4,/test/903
...,...,...,...,...
15677,Thousands of family members and fellow miners ...,None,22,/test/740
15678,The precise number of miners still underground...,None,23,/test/740
15679,Mr. Yildiz said on Tuesday that 787 workers we...,Taner Yildiz,24,/test/740
15680,Turkey ’s disaster management agency estimated...,disaster management agency,25,/test/740


In [ ]:
for idx, sample in tqdm(enumerate(data_model.test_dataset), total=len(data_model.test_dataset)):
    sample = data_model.collate_fn([sample])